In [61]:
from collections import deque

import numpy as np

In [62]:
def parse_grid(file):
    with open(file, 'r') as file_in:
        lines = [line.strip() for line in file_in.readlines()]
        grid = np.array([list(line) for line in lines])

    # Get starting position
    x_start, y_start = np.where(grid == 'S')
    x_start, y_start = x_start[0], y_start[0]

    # Get hash positions
    pos_hash = np.where(grid == '#')
    pos_hash = set([(x, y) for x, y in zip(pos_hash[0], pos_hash[1])])

    return grid, (x_start, y_start), pos_hash

In [63]:
def get_possible_next(current, grid, pos_hash):
    n_rows, n_cols = grid.shape
    (x, y), n_steps = current
    next_n_steps = n_steps + 1
    possible_next = []
    if x > 0:
        possible_next.append(((x - 1, y), next_n_steps))
    if y > 0:
        possible_next.append(((x, y - 1), next_n_steps))
    if x < n_rows - 1:
        possible_next.append(((x + 1, y), next_n_steps))
    if y < n_cols - 1:
        possible_next.append(((x, y + 1), next_n_steps))
    possible_next = [pos for pos in possible_next if pos[0] not in pos_hash]
    return possible_next

In [64]:
def bfs(grid, start, n_steps):
   start = (start, 0)
   frontier = deque([start])
   reached = set()
   reached.add(start)

   while frontier:
      current = frontier.popleft()
      if current[1] >= n_steps:
         continue
      for next in get_possible_next(current, grid, pos_hash):
         if next[0] not in reached:
            frontier.append(next)
            reached.add(next)

   n_reachable = len([node for node in reached if node[1] == n_steps])
   return n_reachable

In [68]:
def count_reachable_positions(grid, start, pos_hash, n_steps):
    n_rows, n_cols = grid.shape
    dp = np.zeros((n_rows, n_cols, n_steps + 1), dtype=int)
    dp[start[0], start[1], 0] = 1

    for step in range(1, n_steps + 1):
        for x in range(n_rows):
            for y in range(n_cols):
                if (x, y) not in pos_hash:
                    dp[x, y, step] = (
                        int(dp[x - 1, y, step - 1] > 0 if x > 0 else 0) or
                        int(dp[x + 1, y, step - 1] > 0 if x < n_rows - 1 else 0) or
                        int(dp[x, y - 1, step - 1] > 0 if y > 0 else 0) or
                        int(dp[x, y + 1, step - 1] > 0 if y < n_cols - 1 else 0)
                    )

    n_reachable = np.sum(dp[:, :, n_steps] > 0)
    return n_reachable

In [74]:
file = 'puzzle.txt'
grid, start, pos_hash = parse_grid(file)
count_reachable_positions(grid, start, pos_hash, n_steps=64)

3642